# Convert MongoDB Output to DataFrames

This notebook takes a mongo database configured for pythia experiments and creates pandas dataframes in order to more easily compare experiments.  The first data frame contains all of the variables from the config and results fields in the database.  The second dataframe, if SAVE_RESULTS is flagged in the experiements, creates fields for each observation.  Error rate for each observation is also calculated.  The ability to export each dataframe as csv is also included. 

In [ ]:
import os
import datetime
import json
import numpy as np
import pandas as pd
import pprint

from IPython.display import display, HTML
from pymongo import MongoClient
%matplotlib inline

The below code connects to MongoDB by allowing you to enter your host, port, and database.

In [ ]:
host = input() #host as string

In [ ]:
port = int(input()) #port as int

In [ ]:
# Connect to Mongo host & port

client = MongoClient(host, port)

In [ ]:
# Input the name of the database you'd like to connect to. Example 'sacred_demo' or 'pythia_experiment'

db_name = input()

In [ ]:
# Connect to db

db = client.get_database(db_name)

The below cell creates a dataframe from the database by combining the 'config' and 'result' fields.  It takes the 2-values fields recall, precision, f score, and support and creates new columns VARIABLE_dup and VARIABLE_nov for each.  This also allows CSV output.  

In [ ]:

cursor = db.default.runs.find()
#count = 0
list_of_dics = []
for doc in cursor:
    if doc['status'] == 'COMPLETED':
        #Create algorithm column with choice of algorithm
        if doc['config']['LOG_REG'] == True:
            doc['config']['algorithms'] = 'LOG_REG'
        elif doc['config']['SVM'] == True:
            doc['config']['algorithms'] = 'SVM'
        else: 
            doc['config']['algorithms'] = 'XGB'

        feature_list = []
        W2V_features = []
        #Add features to feature list to create identifier
        for feature in doc['config'].keys():
            if feature.startswith("MEM"):
                 if doc['config']["MEM_NET"] == True:
                    feature_list.append(feature)

            elif doc['config'][feature] == True and type(doc['config'][feature]) == bool :
                feature_list.append(feature)
            elif feature.startswith("W2V"):
                W2V_features.append(feature + ":" + str(doc['config'][feature]))
            else:
                continue

        # add LDA and WORDONEHOT features to featurelist if they are active in model
        feature_list2 = []
        for item in feature_list:
            if item.startswith("LDA"):
                feature_list2.append("LDA_TOPICS:" + str(doc['config']['LDA_TOPICS']))
            elif item.startswith("WORDONEHOT"):
                feature_list2.append("WORDONEHOT_VOCAB:" + str(doc['config']['WORDONEHOT_VOCAB']))
        full_feature_list = feature_list + feature_list2
        doc['config']["index"] = "_".join(str(e) for e in feature_list)

        #Separate out recall into recall_dup and recall_nov
        combined_dics = doc['config'].copy()     
        doc["result"]["result_dup"] = doc['result']["recall"][0]
        doc["result"]["recall_nov"] = doc['result']["recall"][1]

        doc["result"]["precision_dup"] = doc['result']["precision"][0]
        doc["result"]["precision_nov"] = doc['result']["precision"][1]

        doc["result"]["fscore_dup"] = doc['result']["f score"][0]
        doc["result"]["fscore_nov"] = doc['result']["f score"][1]

        doc["result"]["support_dup"] = doc['result']["support"][0]
        doc["result"]["support_nov"] = doc['result']["support"][1]

        #remove original recall,precision, fscore, and support
        doc["result"].pop("recall")
        doc["result"].pop("precision")
        doc["result"].pop("f score")
        doc["result"].pop("support")


        #combine dictionaries
        combined_dics.update(doc["result"])

        #add to list of dics
        list_of_dics.append(combined_dics)
allvalues_df = pd.DataFrame(list_of_dics)
allvalues_df = allvalues_df.set_index("index")
display(allvalues_df)

Below, save dataframe to CSV

In [ ]:
csv_output = input() #name for csv output

In [ ]:
allvalues_df.to_csv(csv_output)

The below code creates a dataframe that has an observation for each column and an experiment for each row.  Also included is a row for ground truth and error rate for each observation.  

In [ ]:
#This line will delete all rows where the predicted label is null
allvalues_df = allvalues_df[pd.notnull(allvalues_df['predicted_label'])]


In [ ]:

#Create Data frame with observations as columns and models as rows
label_df = pd.DataFrame(allvalues_df['predicted_label'].tolist(), columns=allvalues_df['id'][0])
#Add name as row index
label_df['index'] = allvalues_df.index.values
label_df = label_df.set_index('index')
label_df = label_df.drop('index', 1)

In [ ]:
#Make dataframe for Ground Truth Values
novelty_df = pd.DataFrame(allvalues_df['novelty'][0]).transpose()
novelty_df.columns = list(label_df)
novelty_df['index'] = ['Ground_Truth']
novelty_df = novelty_df.set_index('index')

In [ ]:
#Create combined dataframe with ground truth and labeled predictions for each experiment
labeled_prediction_df = label_df.append(novelty_df)
labeled_prediction_df

Below, the error rate is computed for each observation and added as a row in the dataframe

In [ ]:
error_rate = abs(label_df.sum()/len(label_df) - novelty_df.sum())
error_rate = error_rate.rename("error_rate")
labeled_prediction_df = labeled_prediction_df.append(error_rate)
labeled_prediction_df

The below code reshapes the data to be "tall" where the experiments are the coluns and the observations are rows.  In addition, the ground truth and error rate are columns.

In [ ]:
#make data tall
tall_df = labeled_prediction_df.transpose()



In [ ]:
#print tall df
tall_df

In [ ]:
#Plot a histogram of error rate for each observation
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
plt.figure();
tall_df["error_rate"].hist()

Below, the tall dataframe can be exported to CSV.

In [ ]:
tall_csv_output = input() #name for csv output

In [ ]:
tall_df.to_csv(tall_csv_output) #call to create csv

Below, convert wide dataframe to CSV

In [ ]:
wide_csv_output = input() #name for csv output

In [ ]:
labeled_prediction_df.to_csv(wide_csv_output) #call to create csv